In [1]:
import sys
sys.path.append('../')

In [2]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import model.metric as module_metric

from data_loader.dataloader import get_dataloader
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from model.loss import *
from train import *
from pathlib import Path


In [3]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 4
num_epoch = 100
damage = 'dent'

train_dir = f'./dataset/{damage}/train/'
val_dir = f'./dataset/{damage}/valid/'

In [5]:
transform_train = A.Compose([
    A.HorizontalFlip(),
    A.Rotate((-10, 10), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.Resize(512, 512),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

In [6]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size)

In [7]:
model = smp.Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)
model = model.to(device)

In [8]:
criterion = DiceLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-7)
metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

In [9]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch

train_config['Loss fn'] = criterion.__class__.__name__
train_config['Optimizer'] = optimizer.__class__.__name__
train_config['LR Scheduler'] = scheduler.__class__.__name__
train_config['Metric'] = {str(idx+1) : metric for idx, metric in enumerate([metrics[i].__name__ for i in range(len(metrics))])}


In [10]:
save_dir = f"./saved/U-Net_{model.__dict__['name'].split('-')[1:][0].capitalize()}/{model.__dict__['name'].split('-')[-1].capitalize()}_{damage}_ver0/"

trainer = Trainer(model, criterion, metrics, optimizer, device, num_epoch, save_dir,
                                  data_loader=train_dataloader, valid_data_loader=val_dataloader,
                                  lr_scheduler=scheduler)

In [11]:
wandb.init(project=trainer.dir.split('/')[2], name=f"{trainer.dir.split('/')[3]}", config=train_config)

AttributeError: 'PosixPath' object has no attribute 'split'

In [ ]:
trainer.train()


Epoch : 0 | Train Loss : 0.94079 | Train P.A : 92.46% | Train IOU : 0.01836 | Val Loss : 0.90967 | Val P.A : 94.01% | Val IOU : 0.03732 | Training Time : 94.78sec

Epoch : 1 | Train Loss : 0.88769 | Train P.A : 97.37% | Train IOU : 0.03482 | Val Loss : 0.83314 | Val P.A : 98.39% | Val IOU : 0.04970 | Training Time : 93.21sec

Epoch : 2 | Train Loss : 0.85706 | Train P.A : 98.23% | Train IOU : 0.04137 | Val Loss : 0.88415 | Val P.A : 99.03% | Val IOU : 0.02684 | Training Time : 92.83sec

Epoch : 3 | Train Loss : 0.83099 | Train P.A : 98.40% | Train IOU : 0.04720 | Val Loss : 0.82230 | Val P.A : 98.68% | Val IOU : 0.05442 | Training Time : 92.66sec

Epoch : 4 | Train Loss : 0.82380 | Train P.A : 98.33% | Train IOU : 0.05095 | Val Loss : 0.79360 | Val P.A : 98.61% | Val IOU : 0.05531 | Training Time : 92.71sec

Epoch : 5 | Train Loss : 0.82282 | Train P.A : 98.42% | Train IOU : 0.04970 | Val Loss : 0.80919 | Val P.A : 98.83% | Val IOU : 0.06028 | Training Time : 93.50sec

Epoch : 6 | Tra

KeyboardInterrupt: 